In [1]:
%matplotlib qt


import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from rastermap import Rastermap
from scipy.stats import zscore

# spks is neurons by time
# sys.path.insert(0, '/Users/josefbitzenhofer/Documents/code/viral/data/cached_for_rastermap')
spks_noITI = np.load('/Users/josefbitzenhofer/Documents/code/viral/data/cached_for_rastermap/JB027_2025-02-24_corridor_neur_noITI.npz')["spks"]
spks = np.load('/Users/josefbitzenhofer/Documents/code/viral/data/cached_for_rastermap/JB027_2025-02-24_corridor_neur.npz')["spks"]
behaviour = np.load('/Users/josefbitzenhofer/Documents/code/viral/data/cached_for_rastermap/JB027_2025-02-24_corridor_behavior.npz')

In [2]:
# like Stringer et al. 2024 Fig 2

# fit rastermap
# n_PCs = 200
# n_clusters = 100
# locality = 0.75
# time_lag_windows = 5
model = Rastermap(n_PCs=200, n_clusters=100, 
                  locality=1, time_lag_window=5, time_bin=5).fit(spks_noITI)
# y = model.embedding # neurons x 1
isort = model.isort
cc_nodes = model.cc # sorted asymmetric similarity matrix

X = spks[isort]


2025-02-28 11:02:41,717 [INFO] normalizing data across axis=1
2025-02-28 11:02:41,845 [INFO] binning in time with time_bin = 5
2025-02-28 11:02:42,040 [INFO] projecting out mean along axis=0
2025-02-28 11:02:42,081 [INFO] data normalized, 0.37sec
2025-02-28 11:02:42,081 [INFO] sorting activity: 642 valid samples by 88379 timepoints
2025-02-28 11:02:42,455 [INFO] n_PCs = 200 computed, 0.74sec
2025-02-28 11:02:42,539 [INFO] 93 clusters computed, time 0.83sec


/Users/josefbitzenhofer/Documents/code/viral/.venv/lib/python3.12/site-packages/rastermap/cluster.py:138: UserWarning: found fewer than half the n_clusters that the user specified, rerunning with random initialization
  warnings.warn(
/Users/josefbitzenhofer/Documents/code/viral/.venv/lib/python3.12/site-packages/rastermap/cluster.py:148: UserWarning: found fewer clusters than user specified, try reducing n_clusters and/or reduce n_splits and/or increase n_PCs
  warnings.warn(


2025-02-28 11:02:57,003 [INFO] clusters sorted, time 15.29sec
2025-02-28 11:02:57,061 [INFO] clusters upsampled, time 15.35sec
2025-02-28 11:02:57,189 [INFO] rastermap complete, time 15.48sec


In [3]:
# from Stringer et al. 2024
def bin1d(X, bin_size, axis=0):
    """ mean bin over axis of data with bin bin_size """
    if bin_size > 0:
        size = list(X.shape)
        Xb = X.swapaxes(0, axis)
        size_new = Xb.shape
        Xb = Xb[:size[axis]//bin_size*bin_size].reshape((size[axis]//bin_size, bin_size, *size_new[1:])).mean(axis=1)
        Xb = Xb.swapaxes(axis, 0)
        return Xb
    else:
        return X

In [4]:
# from Stringer et al. 2024
kp_colors = np.array([[0.55,0.55,0.55],
                      [0.,0.,1],
                      [0.8,0,0],
                      [1.,0.4,0.2],
                      [0,0.6,0.4],
                      [0.2,1,0.5],
                      ])

In [5]:
# print(X.shape)
# X_zscored = zscore(X)

In [6]:
# TODO: bin over neurons

bin_size = 5

### ----------- bin across embedding --------------------------------------- ###
        # if data is not None and compute_X_embedding:
        #     bin_size=self.bin_size
        #     if (bin_size==0 or n_samples < bin_size or 
        #         (bin_size == 50 and n_samples < 1000)):
        #         bin_size = max(1, n_samples // 500)
        #     self.X_embedding = zscore(bin1d(X[self.isort], bin_size, axis=0), axis=1)

X_zs = zscore(bin1d(X, bin_size, axis=0), axis=1)

In [7]:
corridor_starts = behaviour["corridor_starts"]
corridor_widths = behaviour["corridor_widths"]
# corridor_imgs = behaviour["corridor_imgs"]
VRpos = behaviour["VRpos"]
reward_idx = behaviour["reward_inds"]
lick_idx = behaviour["lick_inds"]
run = behaviour["run"]

In [8]:
sampling_rate = 30 # fps

In [10]:
n_trials_plot = 25
plot_start = int(corridor_starts[-n_trials_plot, 0])
to_plot = X_zs[:, plot_start:]
nTimepoints = to_plot.shape[1]
xticks = np.arange(0, nTimepoints, sampling_rate * 60) # every minute
xticks_labels = np.arange(0, len(xticks), 1)

lick_idx_plot = lick_idx[lick_idx > plot_start] - plot_start
reward_idx_plot = reward_idx[reward_idx > plot_start] - plot_start

n_features = 4

# Scaling of the rasterplot
vmin = 0.75
vmax = 2

plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 14


fig, ax = plt.subplots(nrows=n_features, ncols=1, figsize=(24, 10), gridspec_kw={"height_ratios": [25, 1, 1, 4]}, sharex=True)

# NEURONS RASTERPLOT
ax1 = ax[0]
ax1.imshow(to_plot, vmin=vmin, vmax=vmax, cmap="gray_r", aspect="auto")
ax1.set_xticks(xticks)
ax1.set_xticklabels(xticks_labels)
ax1.set_xlabel("Time (minutes)")
ax1.set_ylabel("Neurons")

# TODO: y-axis is weird

for (start, reward_condition), width in zip(corridor_starts[-n_trials_plot:, :], corridor_widths[-n_trials_plot:]):
    ax1.fill_betweenx(
        y=[0, to_plot.shape[0]],
        x1=start - plot_start,
        x2=start + width - plot_start,  
        color="forestgreen" if reward_condition == 1 else "violet",
        alpha=0.3
    )


# REWARDS
ax2 = ax[1]
ax2.scatter(
    reward_idx_plot,  # Align with the time window
    np.ones(len(reward_idx_plot)), 
    color="g",
    marker="^", 
    s=30
)
ax2.axis("off")
ax2.legend(["Reward"], loc="center left", frameon=False, handletextpad=0.2)

# LICKING
ax3 = ax[2]
ax3.scatter(
    lick_idx_plot,  # Align with the time window
    np.ones(len(lick_idx_plot)), 
    color=[1.0,0.3,0.3],
    marker=".", 
    s=30
)
ax3.axis("off")
ax3.legend(["Licks"], loc="center left", frameon=False, handletextpad=0.2)


# RUNNING SPEED
ax4 = ax[3]
ax4.fill_between(np.arange(nTimepoints), run[plot_start:], color=kp_colors[0]) #alpha=0.5)
ax4.axis("off")
ax4.legend([matplotlib.lines.Line2D([0], [0], color="none")], ["Running speed"], loc="upper left", frameon=False, handlelength=0, bbox_to_anchor=(0, 1.5))



ax1.set_xlim([0, nTimepoints])
ax2.set_xlim([0, nTimepoints])
ax3.set_xlim([0, nTimepoints])
ax4.set_xlim([0, nTimepoints])

plt.show()

/var/folders/lm/qkbhqz3s6ld73s0twf2bb8_40000gn/T/ipykernel_96203/1489936088.py:25: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax1.imshow(to_plot, vmin=vmin, vmax=vmax, cmap="gray_r", aspect="auto")


ValueError: operands could not be broadcast together with shapes (0,) (58433,) 

2025-02-28 11:04:16.449 python[96203:3472469] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-28 11:04:16.449 python[96203:3472469] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
print(len(to_plot[1]))
print(len(run[plot_start:]))

89186
89186


In [ ]:
print("Reward indices:", reward_idx_plot)
print("Lick indices:", lick_idx_plot)

Reward indices: [  643   644   645   646   647   648   649   650   651  1786  1787  1788
  1789  1790  1791  1792  1793  1794  8272  8273  8274  8275  8276  8277
  8278  8279  8280  9487  9488  9489  9490  9491  9492  9493  9494  9495
 10873 10874 10875 10876 10877 10878 10879 10880 10881 15012 15013 15014
 15015 15016 15017 15018 15019 25530 25531 25532 25533 25534 25535 25536
 25537 25538 26687 26688 26689 26690 26691 26692 26693 26694 28032 28033
 28034 28035 28036 28037 28038 28039 28040 30410 30411 30412 30413 30414
 30415 30416 30417 30418 34594 34595 34596 34597 34598 34599 34600 34601
 34602 36415 36416 36417 36418 36419 36420 36421 36422 36423 47764 47765
 47766 47767 47768 47769 47770 47771 47772 51381 51382 51383 51384 51385
 51386 51387 51388 51389 57272 57273 57274 57275 57276 57277 57278 57279
 57280 58616 58617 58618 58619 58620 58621 58622 58623 58624 60843 60844
 60845 60846 60847 60848 60849 60850 60851 72385 72386 72387 72388 72389
 72390 72391 72392 72393 82475 8247

In [ ]:
# TODO: Add corridor stuff
# TODO: Add licks, rewards, running speed